# Chapter 1: The Python Data Model

Since Python 2.6, namedtuple can be used to build classes of objects that are just bundles of attributes with no custom methods, like a database record.

By implementing the special methods `__len__` and `__getitem__`, our FrenchDeck behaves like a standard Python sequence, allowing it to benefit from core language features (e.g., iteration and slicing) and from the standard library. Thanks to composition, the `__len__` and `__getitem__` implementations can hand off all the work to a list object, `self._cards`

In [41]:
from collections import namedtuple

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = ['spades', 'diamonds', 'clubs', 'hearts']

    def __init__(self):
        self._cards = [Card(rank, suit) for rank in self.ranks 
                                        for suit in self.suits]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

> like any standard Python collection, a deck responds to the len() function by returning the number of cards in it:

In [3]:
deck = FrenchDeck()

In [4]:
len(deck)

52

## `__getitem__`

Implementing the `__getitem__` dunder method allows for FrechDeck elements to be accessed by index, like a list:

In [5]:
deck[0]

Card(rank='2', suit='spades')

In [6]:
deck[-1]

Card(rank='A', suit='hearts')

### Pick a card, any card

Python already has a function to get a random item from a sequence: `random.choice`. We can just use it on a deck instance (thanks to `__getitem__`):

In [38]:
from random import choice
choice(deck)

Card(rank='Q', suit='clubs')

### Slicing

The deck implementation also supports slicing:

In [8]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='2', suit='diamonds'),
 Card(rank='2', suit='clubs')]

In [9]:
deck[12::13]

[Card(rank='5', suit='spades'),
 Card(rank='8', suit='diamonds'),
 Card(rank='J', suit='clubs'),
 Card(rank='A', suit='hearts')]

### It's iterable

Just by implementing the `__getitem__` special method, our deck is also iterable:

In [10]:
for card in deck:
    print(card)

Card(rank='2', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='2', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='clubs')
Card(rank='3', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='clubs')
Card(rank='4', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='clubs')
Card(rank='5', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='clubs')
Card(rank='6', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='clubs')
Card(rank='7', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='clubs')
Card(rank='8', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='clubs')
Card(rank='9', suit='hearts')
Card(rank='10', suit='spades')
C

### Being iterable means `in` operator works without `__contains__`

The `in` operator leverages the `__contains__` implementation of a sequence. When such method is not implemented, it will perform a sequential scan, which means it works in the `FrenchDeck` implementation, even though it is not implementing the `__contains__` method:

In [11]:
Card('Q', 'hearts') in deck

True

In [12]:
Card('7', 'beasts') in deck

False

### Sorting

A common system of ranking cards is by rank (with aces being highest), then by suit in the order of spades (highest), then hearts, diamonds, and clubs (lowest):

In [16]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

A couple of test cases for the algorithm above:

In [40]:
(
  FrenchDeck.ranks.index('A') * len(suit_values) + suit_values['spades'],
  FrenchDeck.ranks.index('2') * len(suit_values) + suit_values['clubs'],
  FrenchDeck.ranks.index('10') * len(suit_values) + suit_values['spades']
)

(51, 0, 35)

The function above applied to the deck sorting:

In [24]:
sorted_deck = sorted(deck, key=spades_high)

In [42]:
(sorted_deck[:3], sorted_deck[-4::])

([Card(rank='2', suit='clubs'),
  Card(rank='2', suit='diamonds'),
  Card(rank='2', suit='hearts')],
 [Card(rank='A', suit='clubs'),
  Card(rank='A', suit='diamonds'),
  Card(rank='A', suit='hearts'),
  Card(rank='A', suit='spades')])

### Shuffling

> As implemented so far, a FrenchDeck cannot be shuffled, because it is immutable: the cards and their positions cannot be changed, except by violating encapsulation and handling the `_cards` attribute directly.

> In Chapter 11, that will be fixed by adding a one-line __setitem__ method.

## How special methods are used

> They are meant to be called by the Python interpreter, and not by you. Normally, your code should not have many direct calls to special methods. Unless you are doing a lot of metaprogramming, you should be implementing special methods more often than invoking them explicitly.

You don’t write `my_object.__len__()`. You write `len(my_object)` and, if `my_object` is an instance of a user-defined class, then Python calls the `__len__` instance method you implemented.

The only special method that is frequently called by user code directly is `__init__`, to _invoke the initializer of the superclass_ in your own `__init__` implementation.

For built-in types like `list`, `str`, `bytearray`, and so on, the interpreter takes a shortcut: the CPython implementation of `len()` actually returns the value of the `ob_size` field in the `PyVarObject` C struct that represents any variable-sized built-in object in memory. This is much faster than calling a method.

More often than not, the special method call is implicit. For example, the statement `for i in x:` actually causes the invocation of `iter(x)`, which in turn may call `x.__iter__()` if that is available.